In [1]:
from pyscf import gto, scf
import scipy
import pyscf.qmmm
import pyscf.dft
import numpy as np
angstrom = 1 / 0.52917721067
from matplotlib import pyplot as plt
from functools import reduce
from numpy.linalg import inv 
# ancora il modo migliore per mettere cariche frazionali sulle molecole, se vai a toccare mol.charges si incazza 
#  quando deve calcolare l' initial guess 

def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords()/angstrom, deltaZ)
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float)
            for i in range (len(q)):
                q[i] += deltaZ[i] 
            return self.mol.energy_nuc(q)
    return(NoSelfQMMM())

def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1]/angstrom)
    dV+=mol.intor('int1e_rinv')*dL[1]
    return dV

In [2]:
molhh=gto.Mole(atom="H 0 0 0 ; H 0 0 .66",basis="sto-3G")
molhh.build()
hf=scf.RHF(molhh)
hfhe=fc(hf,[1.,-1])
hf.scf()
hfhe.scf()

converged SCF energy = -1.11439653848583
converged SCF energy = -2.63149932767394


-2.631499327673943

In [3]:
hf.make_rdm1()

array([[0.58400461, 0.58400461],
       [0.58400461, 0.58400461]])

In [4]:
print(hfhe.make_rdm1())
print(hfhe.mo_coeff)

[[ 2.31345718 -0.23708116]
 [-0.23708116  0.02429588]]
[[ 1.07551317 -0.93449842]
 [-0.11021769  1.42051679]]


In [5]:
S=hf.get_ovlp()
C=hf.mo_coeff
O1=hf.mo_occ
O=np.diag(O1)
P=hf.make_rdm1()
F=hf.get_fock()
h1=hf.get_hcore()
h2=hf.get_veff()
J,K=hf.get_jk()
g_ijkl=hf.mol.intor('int2e_sph')
E=np.diag(hf.mo_energy)
print('COC* = P :', np.allclose(P,C.dot(O.dot(C.T)))) 
print( "J-K/2 = H2 :",np.allclose(h2,J-K/2))
print("J= sum_ij(g_ijklm P_lm ) ",np.allclose(J,np.einsum('ijkl,kl->ij',g_ijkl,P)))
print("K= sum_ik(g_ijklm P_jm ) ",np.allclose(K,np.einsum('ijkl,jl->ik',g_ijkl,P)))

print( "F= H1 +H2 :",np.allclose(F,h1+h2))
print ("numero di elettroni:", S.dot(P).trace())  # numero di elettroni
print("FC=SCE :",np.allclose(np.dot(F,C),S.dot(C.dot(E))))
print("eig(S-1 F) -> C,E ","np.linalg.eig(np.linalg.inv(S).dot(F))" )
print("C*FC = E : ",np.allclose(C.T.dot(F.dot(C))/100,E/100))

COC* = P : True
J-K/2 = H2 : True
J= sum_ij(g_ijklm P_lm )  True
K= sum_ik(g_ijklm P_jm )  True
F= H1 +H2 : True
numero di elettroni: 2.0
FC=SCE : True
eig(S-1 F) -> C,E  np.linalg.eig(np.linalg.inv(S).dot(F))
C*FC = E :  True


In [18]:
print(J)

[[1.37652363 0.98649238]
 [0.98649238 1.37652363]]


$$ J - K/2 =H_2$$
$$H_1+ H_2 = F $$

$$ V_{ij}^{\alpha} = \sum_{kl} (ij|kl)(\gamma_{lk}^\alpha+\gamma_{lk}^\beta)- \sum_{kl} (il|kj) \gamma_{lk}^{\alpha} \\    V_{ij}^{\beta}  = \sum_{kl} (ij|kl)(\gamma_{lk}^{\alpha}+\gamma_{lk}^{\beta})  - \sum_{kl} (il|kj)\gamma_{lk}^\beta $$

Now getting the energies

$$  E = \sum_{ij}h_{ij} \gamma_{ji} + \frac{1}{2}\sum_{ijkl} \gamma_{ji}\gamma_{lk} \langle ik||jl\rangle $$


In [6]:
e_tot=hf.e_tot 
e_nn=hf.energy_nuc() 
e_ne,e_ee=hf.energy_elec()
e_ne-=e_ee

In [7]:
print(e_tot,e_nn+e_ee+e_ne)
print(e_ne,np.trace(P.dot(h1)))
print(e_ee,np.trace(P.dot(h2))/2)

-1.114396538485826 -1.114396538485826
-2.6061863152003397 -2.6061863152003397
0.6900061238054227 0.6900061238054227


Now is to get the alchemical derivatives:
$$ $$

In [8]:
hf0=fc(hf,[.005,-.005])
hf0.scf()
C0=hf0.mo_coeff
P0=hf0.make_rdm1()

hf1=fc(hf,[-.005,.005])
hf1.scf()
C1=hf1.mo_coeff
P1=hf1.make_rdm1()
dV=hf1.get_hcore()-hf.get_hcore()

converged SCF energy = -1.11443873633341
converged SCF energy = -1.11443873633631


In [66]:
dC=(abs(C1)*C/abs(C)-C)
dP=(P1-P)
dF=(hf1.get_fock()-F)
dC

array([[-0.00396422, -0.00161794],
       [ 0.00395934, -0.00162984]])

In [35]:
print(C0)
print(C)
print(abs(C1)*C/abs(C))

[[ 0.54433171 -1.31670758]
 [ 0.53640816  1.31995537]]
[[ 0.54037238 -1.31833743]
 [ 0.54037238  1.31833743]]
[[ 0.53640816 -1.31995537]
 [ 0.54433171  1.31670758]]


In [36]:
print(P0)
print(P)
print(P1)

[[0.59259403 0.58396794]
 [0.58396794 0.57546742]]
[[0.58400461 0.58400461]
 [0.58400461 0.58400461]]
[[0.57546742 0.58396794]
 [0.58396794 0.59259403]]


In [37]:
J1,K1=hf1.get_jk()

In [65]:
dP,dC.dot(O.dot(C.T))+C.dot(O.dot(dC.T))+0.5*dC.dot(O.dot(dC.T)) #last term is 2nd order will neglet

(array([[-8.53718548e-03, -3.66654087e-05],
        [-3.66654087e-05,  8.58942014e-03]]),
 array([[-8.55290051e-03, -2.09697300e-05],
        [-2.09697300e-05,  8.57374379e-03]]))

In [61]:
#dF=dV+dC*h2
print(hf1.get_fock()-F),print(dV+(np.einsum('ijkl,kl->ij',g_ijkl,dP))\
                                  -0.5*np.einsum('ijkl,jl->ik',g_ijkl,dP))
print("------------------------------------")
print(hf1.get_fock()-F),print(dV+(np.einsum('ijkl,kl->ij',g_ijkl,(dC.dot(O.dot(C.T))+C.dot(O.dot(dC.T))))\
                                  -0.5*np.einsum('ijkl,jl->ik',g_ijkl,(dC.dot(O.dot(C.T))+C.dot(O.dot(dC.T))))))


[[ 2.90091574e-03  4.38569015e-06]
 [ 4.38569015e-06 -2.89456664e-03]]
[[ 2.90091574e-03  4.38569015e-06]
 [ 4.38569015e-06 -2.89456664e-03]]
------------------------------------
[[ 2.90091574e-03  4.38569015e-06]
 [ 4.38569015e-06 -2.89456664e-03]]
[[ 2.89087616e-03 -3.80357809e-06]
 [-3.80357809e-06 -2.90460902e-03]]


(None, None)

In [70]:
#dF*C+F*dC=SdCE
dF.dot(C)+F.dot(dC),S.dot(dC.dot(E))

(array([[ 0.00070542, -0.00211254],
        [-0.00069213, -0.00211834]]), array([[ 0.00070132, -0.00211411],
        [-0.0006962 , -0.00211672]]))

In [73]:
U=np.linalg.inv(C).dot(dC)
C.dot(U),dC

(array([[-0.00396422, -0.00161794],
        [ 0.00395934, -0.00162984]]), array([[-0.00396422, -0.00161794],
        [ 0.00395934, -0.00162984]]))

In [75]:
C.T.dot(dF.dot(C)),U.dot(E)-E.dot(U)

(array([[ 4.41521656e-06, -4.12866182e-03],
        [-4.12866182e-03, -4.20994734e-06]]),
 array([[ 0.        , -0.00412863],
        [-0.00412863,  0.        ]]))

In [81]:
C.T.dot(dV.dot(C))

array([[-7.26633707e-13, -3.68589661e-03],
       [-3.68589661e-03,  4.10999835e-13]])

In [82]:
(np.einsum('ijkl,kl->ij',g_ijkl,(dC.dot(O.dot(C.T))+C.dot(O.dot(dC.T))))\
                                  -0.5*np.einsum('ijkl,jl->ik',g_ijkl,(dC.dot(O.dot(C.T))+C.dot(O.dot(dC.T)))))

array([[ 3.03894013e-04, -3.80357741e-06],
       [-3.80357741e-06, -3.17626874e-04]])

In [93]:
C.T.dot(np.einsum('ijkl,kl->ij',g_ijkl,dP).dot(C)),\
np.einsum('ijkl,kl->ij',G_mo,U)

(array([[-2.19024700e-07,  2.12579104e-03],
        [ 2.12579104e-03,  6.29563776e-07]]),
 array([[-6.17657763e-06, -4.98783131e-18],
        [-4.77048956e-18, -6.27921263e-06]]))

In [92]:
G_mo=np.zeros(g_ijkl.shape)
for a in range(2):
    for b in range(2):
        for c in range(2):
            for d in range(2):
                for i in range(2):
                    for j in range(2):
                        for k in range(2):
                            for l in range(2):
                                G_mo[a][b][c][d]+= C[i][a]*C[j][b]*C[k][c]*C[l][d]*g_ijkl[i][j][k][l]
G_mo

array([[[[ 6.90006124e-01,  5.55111512e-17],
         [ 0.00000000e+00,  6.77879609e-01]],

        [[ 1.11022302e-16,  1.76847528e-01],
         [ 1.76847528e-01,  0.00000000e+00]]],


       [[[ 0.00000000e+00,  1.76847528e-01],
         [ 1.76847528e-01, -2.22044605e-16]],

        [[ 6.77879609e-01,  1.11022302e-16],
         [ 3.33066907e-16,  7.12736018e-01]]]])

$$ P+\Delta P = (P+\Delta P)S_N(P+\Delta P)/2$$
At the first order in $\Delta P$
$$(P-PS_NP)/2 + \Delta P = PS_N \Delta P$$
$$ \Delta P = (1-PS_N)^{-1} (PS_NP-P) /2 $$ 
<br>oppure :
$$ \Delta P = (PS_N \Delta P + \Delta PS_NP )/2 $$
se $P=P^\dagger$
$$ \Delta P =\Delta PS_NP$$
$$ \Delta P = \alpha (S_NP-1)^{-1} $$
and from 
$$ tr(PS)=N_e$$
$$ tr(\Delta P S )=0 $$
$$P_{purified} = 3P^2 -2P^3 $$


In [12]:
print(dP)
print((S.dot(dP).trace()))

[[-0.01712661  0.        ]
 [ 0.          0.01712661]]
-4.440892098500626e-16


In [13]:
P,3*P.dot(S.dot(P/2))-2*P.dot((S.dot(P.dot(S.dot(P/4)))))

(array([[0.58400461, 0.58400461],
        [0.58400461, 0.58400461]]), array([[0.58400461, 0.58400461],
        [0.58400461, 0.58400461]]))

In [15]:
dV+dC.dot(h2)

array([[-0.00480544, -0.00580661],
       [ 0.00465664, -0.00171916]])

In [16]:
hf1.get_hcore()-hf.get_hcore()

array([[ 2.58698215e-03, -6.81232848e-13],
       [-6.81232848e-13, -2.58698215e-03]])

In [17]:
dV

array([[ 2.58698215e-03, -6.81232848e-13],
       [-6.81232848e-13, -2.58698215e-03]])